In [7]:
# Atnaujintas variantas, po to, kaip padaryti pakeitimai failų varduose 2022 03 26
#
# Skriptas zive_creat_beats_attrib skirtas zive EKG įrašuose esančių pūpsnių atributų masyvui all_beats_attr
# sukūrti, kad būtų galima juos panaudoti sekų formavimui:
# - atsikratome nepageidaujamų anotacijų: 'U'  ------- ///////////////////// pataisyti, neatsikratyti
# - iš anotacijų suformuojame klasių numerius
# - apskaičiuojami ir į atributus įrašomi RR intervalai: RRl, RRr. Atributų eilutėse,
#  atitinkančios pirmą ir paskutinį pūpsnį, RRl ir RRr reikšmės lygios
#  sentinel = -1  - nereikia, skaičiuojama programiškai /////////////// ///////////////////////

# Pacientų kodų sąrašas SubjCodes (userNr+file_name) paimamas iš info_create_z.json, kurį
# suformuoja zive_create_npy. Masyvas all_beats_attr įrašomas į failą all_beats_attr_z.csv.
# 
import pandas as pd
import numpy as np
from pathlib import Path
from icecream import ic
import json, time, sys

from zive_util_vu import runtime, read_rec_attrib, split_SubjCode, get_annotations_table, print_annotations_table


import warnings
warnings.filterwarnings("ignore")

my_os=sys.platform

print("OS in my system : ",my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'

# Pasiruošimas

# //////////////// NURODOMI PARAMETRAI /////////////////////////////////////////////////////

# Bendras duomenų aplankas, kuriame patalpintas subfolderis name_db

if OS == 'Windows':
    Duomenu_aplankas = 'C:\DI\Data\MIT&ZIVE'   # variantas: Windows
else:
    Duomenu_aplankas = '/home/kesju/DI/Data/MIT&ZIVE'   # arba variantas: UBUNTU, be Docker

# jei variantas Docker pasirenkame:
# Duomenu_aplankas = '/Data/MIT&ZIVE'

#  MIT2ZIVE duomenų aplankas
db_folder = 'ZIVE_BUFFER_2022_03_24'

# Aplankas su MIT2ZIVE EKG įrašais (.npa) ir anotacijomis (.json)
rec_folder = 'records_npy'
all_beats_attr_fname ='all_beats_attr_z.csv'

# Kas kiek išvedamas apdorotų duomenų skaičius
show_period = 100

# Failai pūpsnių klasių formavimui
annot_grouping = {'N':'N', 'S':'S', 'V':'V', 'U':'U'}
selected_beats = {'N':0, 'S':1, 'V':2, 'U':3}

# ///////////////////////////////////////////////////////////////////////////////

# Nuoroda į MIT2ZIVE duomenų aplanką
db_path = Path(Duomenu_aplankas, db_folder)

# Nuoroda į aplanką su MIT2ZIVE EKG įrašais (.npa) ir anotacijomis (.json)
rec_dir = Path(db_path, rec_folder)

# Nuskaitome failą info_create.json ir duomenų rinkinio parametrus
file_path = Path(rec_dir,'info_create_z.json')
with open(file_path) as json_file:
    info_create = json.load(json_file)

fs = info_create['fs'] # diskretizavimo dažnumas
SubjCodes =  info_create['SubjCodes'] # pacientų įrašų sąrašas
annot_list  = info_create['annot_list'] # anotacijų sąrašas
# print(annot_list)

# Susikuriame pagalbinį anotacijų žodyną - dictionary beats_annot
nr_sequence = list(range(13))
beats_annot = dict(zip(annot_list, nr_sequence))

print("\nSkriptas zive_prep skirtas EKG įrašų pjaustymui į užduoto ilgio sekas")
print("\nBendras Zive duomenų aplankas: ", Duomenu_aplankas)
print("Zive EKG įrašų aplankas: ", db_folder)

print("Diskretizavimo dažnis: ", fs)
print("\nPacientų įrašų kodų sąrašas:\n",SubjCodes)
print("\nAnotacijų sąrašas:\n", annot_list)


OS in my system :  linux

Skriptas zive_prep skirtas EKG įrašų pjaustymui į užduoto ilgio sekas

Bendras Zive duomenų aplankas:  /home/kesju/DI/Data/MIT&ZIVE
Zive EKG įrašų aplankas:  ZIVE_BUFFER_2022_03_24
Diskretizavimo dažnis:  200

Pacientų įrašų kodų sąrašas:
 ['1000.1631134.867', '1001.1631076.286', '1002.1630844.567', '1003.1636452.484']

Anotacijų sąrašas:
 ['N', 'S', 'V', 'U']


In [8]:
# I-a dalis. Nustatomas anotacijų pasiskirstymas per visus įrašus.
from zive_util import get_SubjCode

pd.set_option("display.max_rows", 1000, "display.max_columns", 19)
pd.set_option('display.width', 1000)

# Atliekama pirminė EKG įrašų anotacijų analizė

# Pasiruošimas ciklui per pacientų įrašus
labels_rec_all = pd.DataFrame(columns=beats_annot.keys(),dtype=int)
labels_rec_all.insert(0,"SubjCode",0)
labels_rec = []

# Pacientų įrašų sąrašas bandymams (yra dar viena vieta):
# SubjCodes = ['100','101', '230', '231']

# Ciklas per pacientų įrašus

for SubjCode in SubjCodes:
    # Paciento anotacijų nuskaitymas, nustatomas anotacijų 
    # ir užduotų klasių pasiskirstymas per visus įrašus

    labels_rec = np.zeros(labels_rec_all.shape[1],dtype=object)
    labels_rec[0] = str(SubjCode)

    # Nuskaitome anotacijas ir jų indeksus
    file_path = Path(rec_dir, str(SubjCode) + '.json')
    print(file_path)

 #  load data using Python JSON module
    atr_sample, atr_symbol = read_rec_attrib(rec_dir, SubjCode)

    # Ciklas per visas paciento įrašo anotacijas (simbolius)
    for symbol in atr_symbol:
        # Gaunamas anotacijos simbolio numeris anotacijų sąraše
        label = beats_annot.get(symbol)
        if (label == None):
            continue
        labels_rec[label+1] +=1

    # Sumuojame į bendrą masyvą
    # print(labels_rec)
    labels_rec_all.loc[len(labels_rec_all)] = labels_rec

# Ciklo per pacientų įrašus pabaiga

# Parodomas anotacijų ir užduotų klasių pasiskirstymas per visus įrašus
print("\nAnotacijų pasiskirstymas per visus įrašus\n")
print("labels_rec_all:\n")
print(labels_rec_all)
suma = labels_rec_all.iloc[:,1:].sum()
print('\n',' '*5, 'sum: ',str(suma.tolist())[1:-1])
total = suma.sum()
print(' '*5, 'total: ', total)

/home/kesju/DI/Data/MIT&ZIVE/ZIVE_BUFFER_2022_03_24/records_npy/1000.1631134.867.json
/home/kesju/DI/Data/MIT&ZIVE/ZIVE_BUFFER_2022_03_24/records_npy/1001.1631076.286.json
/home/kesju/DI/Data/MIT&ZIVE/ZIVE_BUFFER_2022_03_24/records_npy/1002.1630844.567.json
/home/kesju/DI/Data/MIT&ZIVE/ZIVE_BUFFER_2022_03_24/records_npy/1003.1636452.484.json

Anotacijų pasiskirstymas per visus įrašus

labels_rec_all:

           SubjCode    N   S    V  U
0  1000.1631134.867  656  12    0  0
1  1001.1631076.286  541   0  194  0
2  1002.1630844.567  903   0    0  0
3  1003.1636452.484  600  11    2  0

       sum:  2700, 23, 196, 0
      total:  2919


In [9]:
# II-a dalis. zive EKG įrašai analizuojami, formuojamas freimas all_beats_attr ir  įrašomas į diską. 

# from zive_util import print_annotations_table

print("\nEKG įrašams suformuojamas pūpsnių atributų freimas ir įrašomas į diską")

# Pacientų įrašų sąrašas bandymams:
# SubjCodes = ['10001']


# Sukūriame masyvą sekų atributų sąrašo kaupimui

all_beats_attr = pd.DataFrame(columns= ['userNr', 'file_name', 'sample', 'symbol'])

df = pd.DataFrame({'userNr': pd.Series(dtype='int'),
                   'file_name': pd.Series(dtype='str'),
                   'sample': pd.Series(dtype='int'),
                   'symbol': pd.Series(dtype='str')})
start_time = time.time()

# CIKLAS PER PACIENTŲ ĮRAŠUS

for SubjCode in SubjCodes:
    print("\nPacientas:", SubjCode)
    
    # Paciento anotacijų ir EKG įrašų nuskaitymas, sekų išpjovimas ir įrašymas, sekų atributų formavimas

    # Nuskaitome paciento anotacijas ir jų indeksus
    atr_sample, atr_symbol = read_rec_attrib(rec_dir, SubjCode)

    subject_labels = []
    beat_nr = 0
    icycle = 0

    # Ciklas per visas paciento įrašo anotacijas (simbolius) ir jų vietas (i_sample)
    for i, i_sample in enumerate(atr_sample):
    
        icycle +=1
        if (icycle%show_period == 0):
            print(icycle, end =' ') 

        # Formuojame pūpsnio atributus
        userNr, file_name = split_SubjCode(SubjCode)
        beats_attr = {'userNr':int(userNr), 'file_name':str(file_name), 'sample':int(i_sample), 'symbol':str(atr_symbol[i])}
        all_beats_attr = all_beats_attr.append(beats_attr, ignore_index=True)

        beat_nr +=1


# Ciklo per pacientų įrašus pabaiga

# Atsikratome nepageidaujamų anotacijų: 'U'
# index_names = all_beats_attr[all_beats_attr['symbol'].isin(['U'])].index
# all_beats_attr.drop(index_names, inplace = True)
# print(all_beats_attr.info())

# Pernumeruojame indeksus, kad būtų nuo 0 iš eilės
all_beats_attr.reset_index(inplace = True, drop = True)

# Iš anotacijų suformuojame klasių numerius ir pridedame, kaip naują stulpelį
annot_labels = {key:selected_beats[value] for key, value in annot_grouping.items()}
labels_from_annot = all_beats_attr['symbol'].replace(annot_labels, inplace=False)
all_beats_attr['label'] = labels_from_annot

print("\n")
print("\nVisi duomenys\n")
labels_table, labels_sums = get_annotations_table(all_beats_attr)
print_annotations_table(labels_table, labels_sums, Flag1=True, Flag2=False)

# Įrašome sekos atributų masyvą į seq_dir aplanką
file_path = Path(rec_dir, all_beats_attr_fname)
all_beats_attr.index.name = "id"
print(all_beats_attr.dtypes)
all_beats_attr.to_csv(file_path)
print("\nAtributų freimas įrašytas: į ", file_path, "\n" )

end_time = time.time()
print('\n')
runtime(end_time-start_time)

print("\nPabaiga.............")



EKG įrašams suformuojamas pūpsnių atributų freimas ir įrašomas į diską

Pacientas: 1000.1631134.867
100 200 300 400 500 600 
Pacientas: 1001.1631076.286
100 200 300 400 500 600 700 
Pacientas: 1002.1630844.567
100 200 300 400 500 600 700 800 900 
Pacientas: 1003.1636452.484
100 200 300 400 500 600 


Visi duomenys

symbol               N   S    V   All
SubjCodes                            
1000.1631134.867   656  12    0   668
1001.1631076.286   541   0  194   735
1002.1630844.567   903   0    0   903
1003.1636452.484   600  11    2   613
All               2700  23  196  2919
Total:  5838
userNr       object
file_name    object
sample       object
symbol       object
label         int64
dtype: object

Atributų freimas įrašytas: į  /home/kesju/DI/Data/MIT&ZIVE/ZIVE_BUFFER_2022_03_24/records_npy/all_beats_attr_z.csv 



Runtime: 00:00:49

Pabaiga.............
